In [ ]:
# v4 data

In [1]:
import json
import pprint

def load_jsonl_data(path):
    data = []
    with open(path) as f:
        for l in f:
            d = json.loads(l)
            data.append(d)
            
    return data

hf_eval_data = load_jsonl_data("gorilla/data/apibench/huggingface_eval.json")
len(hf_eval_data), hf_eval_data[0]

(911,
 {'code': "###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.\n###Output: <<<domain>>>: Natural Language Processing Sentence Similarity\n<<<api_call>>>: AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')\n<<<api_provider>>>: Hugging Face Transformers\n<<<explanation>>>:1. We first import the necessary classes and modules from the transformers package. This includes AutoTokenizer and AutoModel for loading the pre-trained models from Hugging Face.\n2. We use the AutoModel.from_pretrained() method to load the 'princeton-nlp/unsup-simcse-roberta-base' model, which is specially designed for calculating sentence similarity.\n3. To build the recommendation feature, we process the text of previously liked articles and compute sentence embeddings. For each new article, we compute its sentence embedding and compare it to the embeddings of previously liked arti

# 1. Instruction

In [2]:
# instruction: apibench - {lib}_train.json - code - instruction

import re

def get_code_parts_from_apibench_data(data):
    text = data['code']
    instruction, _ = text.split("\n###Output")
    
    # Extracting domain, api_call, api_provider, and code using regular expressions
    domain_pattern = r'<<<domain>>>: (.+?)\n'
    api_call_pattern = r'<<<api_call>>>: (.+?)\n'
    api_provider_pattern = r'<<<api_provider>>>: (.+?)\n'
    code_pattern = r'<<<code>>>: (.+)'

    domain = re.search(domain_pattern, text).group(1)
    api_call = re.search(api_call_pattern, text).group(1)
    api_provider = re.search(api_provider_pattern, text).group(1)
    code = re.search(code_pattern, text, re.DOTALL).group(1).strip()

    return {
        'instruction': instruction, 
        'domain': domain, 
        'api_call': api_call, 
        'api_provider': api_provider, 
        'code': code
    }

d = hf_eval_data[0]
code_parts = get_code_parts_from_apibench_data(d)
code_parts, code_parts['instruction']

({'instruction': '###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.',
  'domain': 'Natural Language Processing Sentence Similarity',
  'api_call': "AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')",
  'api_provider': 'Hugging Face Transformers',
  'code': "from transformers import AutoTokenizer, AutoModel\ntokenizer = AutoTokenizer.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')\nmodel = AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')"},
 '###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.')

In [3]:
for d in hf_eval_data:
    code_parts = get_code_parts_from_apibench_data(d)
    print(code_parts['instruction'])
    pprint.pp(d)
    break

###Instruction: Design a feature for a social media website to recommend articles to users based on how similar the articles are to their previously liked articles.
{'code': '###Instruction: Design a feature for a social media website to '
         'recommend articles to users based on how similar the articles are to '
         'their previously liked articles.\n'
         '###Output: <<<domain>>>: Natural Language Processing Sentence '
         'Similarity\n'
         '<<<api_call>>>: '
         "AutoModel.from_pretrained('princeton-nlp/unsup-simcse-roberta-base')\n"
         '<<<api_provider>>>: Hugging Face Transformers\n'
         '<<<explanation>>>:1. We first import the necessary classes and '
         'modules from the transformers package. This includes AutoTokenizer '
         'and AutoModel for loading the pre-trained models from Hugging Face.\n'
         '2. We use the AutoModel.from_pretrained() method to load the '
         "'princeton-nlp/unsup-simcse-roberta-base' model,

# 2. Function / Test Function
- code part -> gpt -> function
- dataset 问题，先通过 prompt 解决一部分，需要对应到 huggingface dataset 名称才能对应
- prompt:
    generate following code based on above infomation:
    1. function with：
    - detailed comments
    - function description
    2. test function with：
    - test dataset
    - using assert in test function
    - do not compare number strictly
    - if dataset is provided in performance - dataset, load the dataset, then select several sample from the dataset, otherwise, using online source, do not leave blank

In [4]:
!pip install openai==0.28.1
!pip install scipy
!pip install tenacity
!pip install tiktoken
!pip install termcolor 
!pip install requests

Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple
Looking in indexes: http://mirrors.aliyun.com/pypi/simple


In [5]:
import os
import openai
import json

openai.api_type = "azure"
openai.api_base = "https://autoagents-ca-east.openai.azure.com"
openai.api_key = "2864ce19a46540b2a0943df607ca6225"
openai.api_version = "2023-12-01-preview"

def get_tool_call_res(prompt, tool):
    response = openai.ChatCompletion.create(
      engine="gpt-4",
      messages=[
        {"role": "system", "content": "You are a helpful assistant designed to output JSON."},
        {"role": "user", "content": prompt}
      ],
      tools = [tool],
      tool_choice="auto"
    )
    args = response.choices[0].message.tool_calls[0].function.arguments
    
    return json.loads(args)

tool = {
  "type": "function",
  "function": {
    "name": "get_current_weather",
    "description": "Get the current weather in a given location",
    "parameters": {
      "type": "object",
      "properties": {
        "location": {
          "type": "string",
          "description": "The city and state, e.g. San Francisco, CA"
        },
        "unit": {
          "type": "string",
          "enum": ["celsius", "fahrenheit"]
        }
      },
      "required": ["location"]
    }
  }
}
res = get_tool_call_res("how is the weather of beijing", tool)
res

{'location': 'Beijing'}

In [44]:
def get_function_from_data(data, err):
    prompt = f"""
**基本信息：**
{data}

**任务要求：**
1. 请设计一个需求，用一句话描述。
2. 根据上述描述，生成实现该需求的代码。    
3. function comments should follow Google Python Style Guide, includes args, returns, and raises
4. 根据生成的代码，编写相应的测试函数。

**测试样例代码：**
1. 函数开始打印 Testing stared
2. 测试用例开始打印 Testing case [x/x] started
3. 函数结束打印 Testing finished
```python
def test_...():
    print("Testing started.")
    dataset = load_dataset("...")
    sample_data = dataset[0]  # 如果是图像数据，从数据集中抽取一个样本

    # 测试用例 1：...
    print("Testing case [1/3] started.")
    assert assert 1, f"Test case [1/3] failed: ..."

    # 测试用例 2：...
    print("Testing case [2/3] started.")
    assert assert 2, f"Test case [2/3] failed: ..."

    # 测试用例 3：...
    print("Testing case [3/3] started.")
    assert assert 3, f"Test case [3/3] failed: ..."
    print("Testing finished.")

# 运行测试函数
test_...()
```"""
    tool = {
        "type": "function",
        "function": {
            "name": "get_codes",
            "description": "生成基于输入的函数代码（function_code）和测试函数代码（test_function_code）。",
            "parameters": {
                "type": "object",
                "properties": {
                    "function_name": {
                        "type": "string",
                        "description": "函数名称"
                    },
                    "function_import": {
                        "type": "string",
                        "description": "在函数代码之前导入必要的库"
                    },
                    "function_code": {
                        "type": "string",
                        "description": "独立的函数代码定义和功能的注释、代码实现、分步注释"
                    },
                    "test_function_code": {
                        "type": "string",
                        "description": "独立的测试函数代码，不包含调用代码，参照测试样例的代码"
                    },
                    "call_test_function_line": {
                        "type": "string",
                        "description": "调用测试函数的代码行"
                    },
                    "requirements_install": {
                        "type": "string",
                        "description": "安装必要的依赖，使用最新的，e.g. !pip install -U package_1 package_2"
                    }
                },
                "required": ["function_name", "function_import", "function_code", "test_function_code", "call_test_function_line", "requirements_install"]
            }
        }
    }
    res = get_tool_call_res(prompt, tool)
    return res

res = get_function_from_data(hf_eval_data[1], "")
res

{'function_name': 'extract_medical_embeddings',
 'function_import': 'from transformers import AutoTokenizer, AutoModel',
 'function_code': 'def extract_medical_embeddings(medical_term: str) -> torch.Tensor:\n    """\n    Extract embedding vectors for a medical term using a pre-trained model.\n\n    Args:\n        medical_term (str): The medical term to be converted into an embedding vector.\n\n    Returns:\n        torch.Tensor: A tensor representing the embedding of the medical term.\n\n    Raises:\n        ValueError: If the medical_term is empty.\n    """\n    if not medical_term:\n        raise ValueError(\'The medical term cannot be empty.\')\n    tokenizer = AutoTokenizer.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n    model = AutoModel.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n    inputs = tokenizer(medical_term, return_tensors=\'pt\')\n    outputs = model(**inputs)\n    embeddings = outputs.last_hidden_state\n    return embeddings',
 'test_function_code': "def test_extrac

In [46]:
print(hf_eval_data[1])
print("----------------------")
print(res['requirements_install'])
print("----------------------")
print(res['function_import'])
print("----------------------")
print(res['function_code'])
print("----------------------")
print(res['test_function_code'])
print("----------------------")
print(res['call_test_function_line'])

{'code': '###Instruction: The user is interested in a tool to find relationships between medical terms.\n###Output: <<<domain>>>: Multimodal Feature Extraction\n<<<api_call>>>: AutoModel.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\n<<<api_provider>>>: Hugging Face Transformers\n<<<explanation>>>: 1. We import the necessary classes from the transformers package provided by Hugging Face.\n2. We then call the "AutoModel.from_pretrained" method with the argument \'GanjinZero/UMLSBert_ENG\' to load this pretrained model.\n3. This model, which is particularly suitable for finding relationships between medical terms, can be used to convert medical terms into embeddings (dense vectors).\n4. These embeddings can then be compared to find similarities and relationships between various medical terms.\n<<<code>>>: from transformers import AutoTokenizer, AutoModel\ntokenizer = AutoTokenizer.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\nmodel = AutoModel.from_pretrained(\'GanjinZero/UMLSBert_ENG\')\

In [ ]:
import traceback
import glob
import os

# err_dir = "output/hf-eval-data-v2"
output_dir = "output/hf-eval-data-v4_2"

try:
    os.mkdir(output_dir)
except:
    pass
    

for idx, d in enumerate(hf_eval_data):
    print(idx + 1, end="...")
    formatted_number = str(idx + 1).zfill(5)
    
    # 跳过已生成的
    matching_files = glob.glob(f"{output_dir}/f{formatted_number}_*")
    if matching_files:
        print("skip", end="...")
        continue
    
    max_retries = 3
    retry_count = 0

    while retry_count < max_retries:
        try:
            # err = get_v2_err(err_dir, idx + 1)
            resp = get_function_from_data(d, "")

            # 写入 prompt
            with open(f"{output_dir}/f{formatted_number}_{resp['function_name']}.prompt", 'w') as f:
                f.write(str(d) + "\n\n")
                # f.write(str(err))

            # 写入 python
            with open(f"{output_dir}/f{formatted_number}_{resp['function_name']}.py", 'w') as f:
                f.write("# requirements_file --------------------\n\n")
                f.write(resp['requirements_install'])
                
                f.write("\n\n# function_import --------------------\n\n")
                f.write(resp['function_import'])

                f.write("\n\n# function_code --------------------\n\n")
                f.write(resp['function_code'])

                f.write("\n\n# test_function_code --------------------\n\n")
                f.write(resp['test_function_code'])
                
                f.write("\n\n# call_test_function_line --------------------\n\n")
                f.write(resp['call_test_function_line'])

            break  # Break out of the loop if the operation is successful
        except Exception:
            retry_count += 1
            if retry_count < max_retries:
                print(f"Retrying... (Attempt {retry_count}/{max_retries})", end="...")
            else:
                print("Max retries reached. Exiting.", end="...")
                traceback.print_exc()


1...2...3...4...Retrying... (Attempt 1/3)...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...58...59...60...61...62...63...64...65...66...67...68...69...70...71...72...73...74...75...76...77...78...79...80...81...82...83...84...85...86...87...88...89...90...91...92...93...94...95...96...97...98...99...100...101...102...103...104...105...106...107...108...109...110...111...112...113...114...115...116...117...118...Retrying... (Attempt 1/3)...119...120...121...122...123...124...125...126...127...128...129...130...131...132...133...134...135...136...137...138...139...140...141...142...143...144...145...146...147...148...149...150...151...152...153...154...155...156...157...158...159...160...161...162...163...164...165...166...167...168...169...170...171...172...173...174...175...17

In [85]:
import glob
import json
import os


# 指定要遍历的目录
directory_path = 'output/hf-eval-data-v4_2'
output_path = 'output/hf-eval-data-v4_3'

try:
    os.mkdir(output_path)
except:
    pass

# 使用 glob.glob 获取所有 .py 文件的列表
python_files = glob.glob(f'{directory_path}/*.py')

# 遍历文件列表
for idx, file_path in enumerate(python_files):
    print(idx, end="...")

    # 在这里可以对每个 .py 文件执行需要的操作
    # 例如，读取文件内容、分析代码等
    
    file_content = ""
    with open(file_path, 'r') as file:
        file_content = file.read()
        
    # print(file_content)
    # req line
    pip_line_idx = -1
    for idx, l in enumerate(file_content.split("\n")):
        if "!pip install" in l:
            packages = l.split("!pip install -U ")[1].split(" ")
            new_packages = []
            
            for p in packages:
                if "==" in p:
                    p = p.split("==")[0]
                    print("!",end='...')
                new_packages.append(p)
                
            packages_json = json.dumps(new_packages)
            
            prog = \
f"""
import subprocess

requirements = {packages_json}

for package in requirements:
    subprocess.run(['pip', 'install', '-U', package])
"""
            
            # print(prog)
            pip_line_idx = idx
            break
            
    if pip_line_idx != -1:
        content_lines = file_content.split("\n")
        final_prog = "\n".join(content_lines[:pip_line_idx]) + prog + "\n".join(content_lines[pip_line_idx+1:])
        # print(final_prog)
        
        # print(output_path + "/" + file_path.split("/")[-1])
        with open(output_path + "/" + file_path.split("/")[-1], 'w') as f:
            f.write(final_prog)

0...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...!...!...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...49...50...51...52...53...54...55...56...57...58...59...60...61...62...63...64...65...66...67...68...69...70...71...72...73...74...75...76...77...78...79...80...81...82...83...84...85...86...87...88...89...90...91...92...93...94...95...96...97...98...99...!...!...100...101...102...103...104...!...!...!...105...106...107...108...109...110...111...112...113...114...115...116...117...118...119...120...121...122...123...124...125...126...127...128...129...130...131...132...133...134...135...136...137...138...139...140...141...142...143...144...145...146...!...!...!...!...147...148...149...150...151...152...153...154...155...156...157...158...159...160...161...162...163...164...165...166...167...168...169...170...171...172...173...174...175...176...177.

# Analyze

In [30]:
# output 中的结果，检查一下有没有 Test Finished 的

import glob
import shutil

output_dir = "output/hf-eval-data-v4-valid-result/codellama-13b-python-eval/"

results = []
valid_paths = []

python_files = glob.glob(f'{output_dir}/*.py')

# 遍历文件列表
for idx, file_path in enumerate(python_files):
    print(idx, end="...")
    # print(file_path)
    
    err_file = file_path.split(".")[0] + ".err"
    
    out_file = file_path.split(".")[0] + ".out"
    
    stats = ["", [], ""]
    
    try:
        with open(out_file) as f:
            for l in f:
                l = l.strip()

                if "Testing started" in l:
                    stats[0] = l
                if "Testing case" in l:
                    stats[1].append(l)
                if "Testing finish" in l:
                    stats[2] = l
                    valid_paths.append(file_path)
    
        # print(stats)

        results.append(stats)
    except:
        pass

0...1...2...3...4...5...6...7...8...9...10...11...12...13...14...15...16...17...18...19...20...21...22...23...24...25...26...27...28...29...30...31...32...33...34...35...36...37...38...39...40...41...42...43...44...45...46...47...48...

In [31]:
start_cnt = 0
case1_start = 0
case2_start = 0
case3_start = 0
finish_cnt = 0

for start, cases, finish in results:
    if start != '':
        start_cnt += 1
    if len(cases) == 1:
        case1_start += 1
    if len(cases) == 2:
        case2_start += 1
    if len(cases) == 3:
        case3_start += 1
    if finish != '':
        finish_cnt += 1

print(start_cnt, start_cnt / len(results))
print(case1_start, case1_start / len(results))
print(case2_start, case2_start / len(results))
print(case3_start, case3_start / len(results))
print(finish_cnt, finish_cnt / len(results))

32 0.6530612244897959
26 0.5306122448979592
2 0.04081632653061224
2 0.04081632653061224
8 0.16326530612244897


In [14]:
len(results)

887

In [6]:
# valid 输出到一个文件夹里
import os
import shutil

target_dir = "output/hf-eval-data-v4-valid"
try:
    os.mkdir(target_dir)
except:
    pass

for v in valid_paths:
    shutil.copy(v, target_dir + "/" + v.split("/")[-1])